### Import Libraries

In [1]:
import pandas as pd
import requests

In [2]:
pd.set_option('display.max_columns', 30)

### Helper Functions

In [3]:
def get_json_from_url(url, headers=None, timeout=10):
    response = requests.get(url, headers=headers,timeout=timeout)
    response.raise_for_status() 
    
    return response.json()

### Read Stations Info

In [4]:
stations_info_data = get_json_from_url("https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information")

In [5]:
stations_info = stations_info_data.get('data', {}).get('stations', [])

In [6]:
len(stations_info)

863

In [7]:
stations_info_df = pd.DataFrame(stations_info)

In [8]:
stations_info_df.head()

station_id                          name physical_configuration        lat  \
0       7000  Fort York  Blvd / Capreol Ct                REGULAR  43.639832   
1       7001     Wellesley Station Green P    ELECTRICBIKESTATION  43.664964   
2       7002    St. George St / Bloor St W                REGULAR  43.667131   
3       7003      Madison Ave / Bloor St W                REGULAR  43.667018   
4       7005           King St W / York St                REGULAR  43.648001   

         lon  altitude                       address  capacity  \
0 -79.395954       NaN  Fort York  Blvd / Capreol Ct        47   
1 -79.383550       NaN             Yonge / Wellesley        23   
2 -79.399555       NaN    St. George St / Bloor St W        17   
3 -79.402796       NaN      Madison Ave / Bloor St W        15   
4 -79.383177       0.0           King St W / York St        21   

   is_charging_station                         rental_methods  \
0                False  [KEY, TRANSITCARD, CREDITCARD, PHONE]   
1                 True  [KEY, TRANSITCARD, CREDITCARD, PHONE]   
2                False  [KEY, TRANSITCARD, CREDITCARD, PHONE]   
3                False  [KEY, TRANSITCARD, CREDITCARD, PHONE]   
4                False  [KEY, TRANSITCARD, CREDITCARD, PHONE]   

                                              groups          obcn  \
0        [South, Fort York - Entertainment District]  647-643-9607   
1  [E-Charging , South, Church Wellesley / Yorkvi...  416-617-9576   
2                     [South, U of T - Hospital Row]  647-643-9615   
3                        [South, Bloor St W / Annex]  647-631-4587   
4                        [South, Financial District]  647-643-9693   

     short_name  nearby_distance  _ride_code_support rental_uris post_code  \
0  647-643-9607            500.0                True          {}       NaN   
1  416-617-9576            500.0                True          {}   M4Y 1G7   
2  647-643-9615            500.0                True          {}       NaN   
3  647-631-4587            500.0                True          {}       NaN   
4  647-643-9693            500.0                True          {}       NaN   

  is_valet_station cross_street  
0              NaN          NaN  
1              NaN          NaN  
2              NaN          NaN  
3              NaN          NaN  
4              NaN          NaN

In [9]:
len(stations_info_df)

863

### Read Stations Status

In [10]:
stations_status_data = get_json_from_url("https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_status")

In [11]:
stations_status = stations_status_data.get('data', {}).get('stations', [])

In [12]:
len(stations_status)

863

In [13]:
stations_status_df = pd.DataFrame(stations_status)

In [14]:
stations_status_df.head()

station_id  num_bikes_available       num_bikes_available_types  \
0       7000                   22  {'mechanical': 21, 'ebike': 1}   
1       7001                    9   {'mechanical': 8, 'ebike': 1}   
2       7002                    3   {'mechanical': 3, 'ebike': 0}   
3       7003                    4   {'mechanical': 4, 'ebike': 0}   
4       7005                   15  {'mechanical': 15, 'ebike': 0}   

   num_bikes_disabled  num_docks_available  num_docks_disabled  last_reported  \
0                   0                   24                   1     1739749987   
1                   0                   14                   0     1739749907   
2                   0                   13                   1     1739749915   
3                   0                   11                   0     1739749991   
4                   0                    6                   0     1739749966   

   is_charging_station      status  is_installed  is_renting  is_returning  \
0                False  IN_SERVICE             1           1             1   
1                 True  IN_SERVICE             1           1             1   
2                False  IN_SERVICE             1           1             1   
3                False  IN_SERVICE             1           1             1   
4                False  IN_SERVICE             1           1             1   

  traffic  
0    None  
1    None  
2    None  
3    None  
4    None

### Data Transformation

#### 1. Join Stations Info and Stations Status 

In [15]:
merged_df = pd.merge(stations_info_df, stations_status_df, on="station_id", how="inner")

In [16]:
merged_df.head()

station_id                          name physical_configuration        lat  \
0       7000  Fort York  Blvd / Capreol Ct                REGULAR  43.639832   
1       7001     Wellesley Station Green P    ELECTRICBIKESTATION  43.664964   
2       7002    St. George St / Bloor St W                REGULAR  43.667131   
3       7003      Madison Ave / Bloor St W                REGULAR  43.667018   
4       7005           King St W / York St                REGULAR  43.648001   

         lon  altitude                       address  capacity  \
0 -79.395954       NaN  Fort York  Blvd / Capreol Ct        47   
1 -79.383550       NaN             Yonge / Wellesley        23   
2 -79.399555       NaN    St. George St / Bloor St W        17   
3 -79.402796       NaN      Madison Ave / Bloor St W        15   
4 -79.383177       0.0           King St W / York St        21   

   is_charging_station_x                         rental_methods  \
0                  False  [KEY, TRANSITCARD, CREDITCARD, PHONE]   
1                   True  [KEY, TRANSITCARD, CREDITCARD, PHONE]   
2                  False  [KEY, TRANSITCARD, CREDITCARD, PHONE]   
3                  False  [KEY, TRANSITCARD, CREDITCARD, PHONE]   
4                  False  [KEY, TRANSITCARD, CREDITCARD, PHONE]   

                                              groups          obcn  \
0        [South, Fort York - Entertainment District]  647-643-9607   
1  [E-Charging , South, Church Wellesley / Yorkvi...  416-617-9576   
2                     [South, U of T - Hospital Row]  647-643-9615   
3                        [South, Bloor St W / Annex]  647-631-4587   
4                        [South, Financial District]  647-643-9693   

     short_name  nearby_distance  _ride_code_support  ... post_code  \
0  647-643-9607            500.0                True  ...       NaN   
1  416-617-9576            500.0                True  ...   M4Y 1G7   
2  647-643-9615            500.0                True  ...       NaN   
3  647-631-4587            500.0                True  ...       NaN   
4  647-643-9693            500.0                True  ...       NaN   

  is_valet_station cross_street num_bikes_available  \
0              NaN          NaN                  22   
1              NaN          NaN                   9   
2              NaN          NaN                   3   
3              NaN          NaN                   4   
4              NaN          NaN                  15   

        num_bikes_available_types num_bikes_disabled  num_docks_available  \
0  {'mechanical': 21, 'ebike': 1}                  0                   24   
1   {'mechanical': 8, 'ebike': 1}                  0                   14   
2   {'mechanical': 3, 'ebike': 0}                  0                   13   
3   {'mechanical': 4, 'ebike': 0}                  0                   11   
4  {'mechanical': 15, 'ebike': 0}                  0                    6   

   num_docks_disabled  last_reported  is_charging_station_y      status  \
0                   1     1739749987                  False  IN_SERVICE   
1                   0     1739749907                   True  IN_SERVICE   
2                   1     1739749915                  False  IN_SERVICE   
3                   0     1739749991                  False  IN_SERVICE   
4                   0     1739749966                  False  IN_SERVICE   

  is_installed  is_renting  is_returning  traffic  
0            1           1             1     None  
1            1           1             1     None  
2            1           1             1     None  
3            1           1             1     None  
4            1           1             1     None  

[5 rows x 31 columns]

In [17]:
len(merged_df)

863

#### 2. Remove stations not in service

In [18]:
print(merged_df['status'].unique())

['IN_SERVICE']


In [19]:
filtered_df = merged_df[merged_df['status'] == 'IN_SERVICE']

In [20]:
len(filtered_df)

863

#### 3. Apply Transformations

In [21]:
filtered_df.dtypes

station_id                    object
name                          object
physical_configuration        object
lat                          float64
lon                          float64
altitude                     float64
address                       object
capacity                       int64
is_charging_station_x           bool
rental_methods                object
groups                        object
obcn                          object
short_name                    object
nearby_distance              float64
_ride_code_support              bool
rental_uris                   object
post_code                     object
is_valet_station              object
cross_street                  object
num_bikes_available            int64
num_bikes_available_types     object
num_bikes_disabled             int64
num_docks_available            int64
num_docks_disabled             int64
last_reported                  int64
is_charging_station_y           bool
status                        object
i

In [22]:
# Add last_updated column from station_status API response

last_updated = stations_status_data['last_updated']
filtered_df['last_updated'] = pd.to_datetime(last_updated, unit='s')
filtered_df.head()

station_id                          name physical_configuration        lat  \
0       7000  Fort York  Blvd / Capreol Ct                REGULAR  43.639832   
1       7001     Wellesley Station Green P    ELECTRICBIKESTATION  43.664964   
2       7002    St. George St / Bloor St W                REGULAR  43.667131   
3       7003      Madison Ave / Bloor St W                REGULAR  43.667018   
4       7005           King St W / York St                REGULAR  43.648001   

         lon  altitude                       address  capacity  \
0 -79.395954       NaN  Fort York  Blvd / Capreol Ct        47   
1 -79.383550       NaN             Yonge / Wellesley        23   
2 -79.399555       NaN    St. George St / Bloor St W        17   
3 -79.402796       NaN      Madison Ave / Bloor St W        15   
4 -79.383177       0.0           King St W / York St        21   

   is_charging_station_x                         rental_methods  \
0                  False  [KEY, TRANSITCARD, CREDITCARD, PHONE]   
1                   True  [KEY, TRANSITCARD, CREDITCARD, PHONE]   
2                  False  [KEY, TRANSITCARD, CREDITCARD, PHONE]   
3                  False  [KEY, TRANSITCARD, CREDITCARD, PHONE]   
4                  False  [KEY, TRANSITCARD, CREDITCARD, PHONE]   

                                              groups          obcn  \
0        [South, Fort York - Entertainment District]  647-643-9607   
1  [E-Charging , South, Church Wellesley / Yorkvi...  416-617-9576   
2                     [South, U of T - Hospital Row]  647-643-9615   
3                        [South, Bloor St W / Annex]  647-631-4587   
4                        [South, Financial District]  647-643-9693   

     short_name  nearby_distance  _ride_code_support  ... is_valet_station  \
0  647-643-9607            500.0                True  ...              NaN   
1  416-617-9576            500.0                True  ...              NaN   
2  647-643-9615            500.0                True  ...              NaN   
3  647-631-4587            500.0                True  ...              NaN   
4  647-643-9693            500.0                True  ...              NaN   

  cross_street num_bikes_available       num_bikes_available_types  \
0          NaN                  22  {'mechanical': 21, 'ebike': 1}   
1          NaN                   9   {'mechanical': 8, 'ebike': 1}   
2          NaN                   3   {'mechanical': 3, 'ebike': 0}   
3          NaN                   4   {'mechanical': 4, 'ebike': 0}   
4          NaN                  15  {'mechanical': 15, 'ebike': 0}   

   num_bikes_disabled num_docks_available  num_docks_disabled  last_reported  \
0                   0                  24                   1     1739749987   
1                   0                  14                   0     1739749907   
2                   0                  13                   1     1739749915   
3                   0                  11                   0     1739749991   
4                   0                   6                   0     1739749966   

   is_charging_station_y      status  is_installed is_renting  is_returning  \
0                  False  IN_SERVICE             1          1             1   
1                   True  IN_SERVICE             1          1             1   
2                  False  IN_SERVICE             1          1             1   
3                  False  IN_SERVICE             1          1             1   
4                  False  IN_SERVICE             1          1             1   

   traffic        last_updated  
0     None 2025-02-16 23:53:30  
1     None 2025-02-16 23:53:30  
2     None 2025-02-16 23:53:30  
3     None 2025-02-16 23:53:30  
4     None 2025-02-16 23:53:30  

[5 rows x 32 columns]

In [23]:
# Create mechanical_bikes_available and electric_bikes_available features

def extract_mechanical_bikes(bike_types): 
    return bike_types.get('mechanical', 0) 

def extract_electric_bikes(bike_types):
    return bike_types.get('ebike', 0) 

filtered_df['mechanical_bikes_available'] = filtered_df['num_bikes_available_types'].apply(extract_mechanical_bikes)
filtered_df['electric_bikes_available'] = filtered_df['num_bikes_available_types'].apply(extract_electric_bikes)


In [24]:
# Convert is_charging_station column to Binary
filtered_df['is_charging_station']  = filtered_df['is_charging_station_x'].map({True: 1, False: 0})

In [25]:
# Convert rental_methods and groups to comma separated text keywords.

filtered_df['rental_methods'] = filtered_df['rental_methods'].apply(lambda data: ",".join([f"'{keyword}'" for keyword in data]))
filtered_df['groups'] = filtered_df['groups'].apply(lambda data: ",".join([f"'{keyword}'" for keyword in data]))

In [26]:
# Format last_reported to 'YYYY-MM-DD HH:MIN:SEC'
filtered_df['last_reported'] = pd.to_datetime(filtered_df['last_reported'], unit='s')


In [27]:
# Select columns for target csv
df_cleaned = filtered_df[[
    'last_updated', 'station_id', 'name', 'physical_configuration', 
    'lat', 'lon', 'altitude',
    'address', 'capacity', 'rental_methods', 'groups', 'obcn', 'nearby_distance', 
    'num_bikes_available', 'mechanical_bikes_available', 'electric_bikes_available',
    'num_bikes_disabled', 'num_docks_available', 'num_docks_disabled', 
    'is_installed', 'is_renting', 'is_returning', 'last_reported', 'is_charging_station', 'status'
]]


In [28]:
# Rename columns for target csv
df_cleaned.columns = [
    'last_update', 'station_id', 'station_name', 'physical_configuration', 
    'latitude', 'longitude', 'altitude', 
    'address', 'capacity', 'rental_methods', 'groups', 'obcn', 'nearby_distance', 
    'num_bikes_available', 'mechanical_bikes_available', 'electric_bikes_available', 
    'num_bikes_disabled', 'num_docks_available', 'num_docks_disabled', 
    'is_installed', 'is_renting', 'is_returning', 'last_reported', 'is_charging_station', 'status'
]

In [29]:
df_cleaned.head()

last_update station_id                  station_name  \
0 2025-02-16 23:53:30       7000  Fort York  Blvd / Capreol Ct   
1 2025-02-16 23:53:30       7001     Wellesley Station Green P   
2 2025-02-16 23:53:30       7002    St. George St / Bloor St W   
3 2025-02-16 23:53:30       7003      Madison Ave / Bloor St W   
4 2025-02-16 23:53:30       7005           King St W / York St   

  physical_configuration   latitude  longitude  altitude  \
0                REGULAR  43.639832 -79.395954       NaN   
1    ELECTRICBIKESTATION  43.664964 -79.383550       NaN   
2                REGULAR  43.667131 -79.399555       NaN   
3                REGULAR  43.667018 -79.402796       NaN   
4                REGULAR  43.648001 -79.383177       0.0   

                        address  capacity  \
0  Fort York  Blvd / Capreol Ct        47   
1             Yonge / Wellesley        23   
2    St. George St / Bloor St W        17   
3      Madison Ave / Bloor St W        15   
4           King St W / York St        21   

                             rental_methods  \
0  'KEY','TRANSITCARD','CREDITCARD','PHONE'   
1  'KEY','TRANSITCARD','CREDITCARD','PHONE'   
2  'KEY','TRANSITCARD','CREDITCARD','PHONE'   
3  'KEY','TRANSITCARD','CREDITCARD','PHONE'   
4  'KEY','TRANSITCARD','CREDITCARD','PHONE'   

                                              groups          obcn  \
0       'South','Fort York - Entertainment District'  647-643-9607   
1  'E-Charging ','South','Church Wellesley / York...  416-617-9576   
2                    'South','U of T - Hospital Row'  647-643-9615   
3                       'South','Bloor St W / Annex'  647-631-4587   
4                       'South','Financial District'  647-643-9693   

   nearby_distance  num_bikes_available  mechanical_bikes_available  \
0            500.0                   22                          21   
1            500.0                    9                           8   
2            500.0                    3                           3   
3            500.0                    4                           4   
4            500.0                   15                          15   

   electric_bikes_available  num_bikes_disabled  num_docks_available  \
0                         1                   0                   24   
1                         1                   0                   14   
2                         0                   0                   13   
3                         0                   0                   11   
4                         0                   0                    6   

   num_docks_disabled  is_installed  is_renting  is_returning  \
0                   1             1           1             1   
1                   0             1           1             1   
2                   1             1           1             1   
3                   0             1           1             1   
4                   0             1           1             1   

        last_reported  is_charging_station      status  
0 2025-02-16 23:53:07                    0  IN_SERVICE  
1 2025-02-16 23:51:47                    1  IN_SERVICE  
2 2025-02-16 23:51:55                    0  IN_SERVICE  
3 2025-02-16 23:53:11                    0  IN_SERVICE  
4 2025-02-16 23:52:46                    0  IN_SERVICE

In [30]:
len(df_cleaned)

863

### Write to CSV

In [31]:
df_cleaned.to_csv('stations.csv', index=False)